# Hybrid digital twin of a Li-ion battery

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
from matplotlib import pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

## 1. Load experimental data

In [2]:
df = pd.read_csv('discharge.csv')

In [18]:
df.head()

,Voltage_measured,Current_measured,Temperature_measured,Current_charge,Voltage_charge,Time,Capacity,id_cycle,type,ambient_temperature,time,Battery
154,3.382031,-2.012391,36.068027,1.9982,2.466,2897.937,1.856487,1,discharge,24,2008.0,B0005
155,3.376548,-2.011771,36.151799,1.9982,2.460,2917.281,1.856487,1,discharge,24,2008.0,B0005
156,3.370452,-2.013057,36.226208,1.9982,2.454,2936.687,1.856487,1,discharge,24,2008.0,B0005
157,3.363787,-2.013130,36.317584,1.9982,2.448,2956.078,1.856487,1,discharge,24,2008.0,B0005
158,3.356852,-2.010505,36.401753,1.9982,2.442,2975.453,1.856487,1,discharge,24,2008.0,B0005


In [3]:
df = df[df['Battery'] == 'B0005']
df = df[df['Temperature_measured'] > 36] #choose battery B0005
#df['Time'] =pd.to_datetime(df['Time'], unit='s')
dfb = df.groupby(['id_cycle']).max()
dfb['Cumulated_T'] = dfb['Time'].cumsum()

In [19]:
dfb.head()

,Voltage_measured,Current_measured,Temperature_measured,Current_charge,Voltage_charge,Time,Capacity,type,ambient_temperature,time,Battery,Cumulated_T,C. Capacity
id_cycle,,,,,,,,,,,,,
1,3.382031,-2.010069,38.904112,1.9984,2.466,3346.937,1.856487,discharge,24,2008.0,B0005,3346.937,1.853684
2,3.402555,-2.009236,38.934590,1.9984,2.475,3328.828,1.846327,discharge,24,2008.0,B0005,6675.765,1.850850
3,3.398392,-2.008818,38.743649,1.9984,2.466,3309.422,1.835349,discharge,24,2008.0,B0005,9985.187,1.848030
4,3.387662,-2.009841,38.672511,1.9982,2.454,3309.719,1.835263,discharge,24,2008.0,B0005,13294.906,1.845242
5,3.381617,-2.009336,38.558828,1.9984,2.449,3307.688,1.834646,discharge,24,2008.0,B0005,16602.594,1.842473


In [4]:
import plotly.express as px
fig = px.scatter_matrix(dfb.drop(columns=['Time','type', 'ambient_temperature', 
                                          'time', 'Battery']), 
                                )
fig.update_traces(marker=dict(size=2,
                              color='crimson',
                              symbol='square')),
fig.update_traces(diagonal_visible=False)
fig.update_layout(
    title='Battery dataset',
    width=900,
    height=1200,
)
fig.update_layout({'plot_bgcolor': '#f2f8fd',
                   'paper_bgcolor': 'white',}, 
                    template='plotly_white',
                    font=dict(size=7)
                    )

fig.show()

In [5]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=dfb['Cumulated_T']/3600, 
                         y=dfb['Capacity'],
                         mode='lines',
                         name='Capacity',
                         marker_size=3, 
                         line=dict(color='crimson', width=3)    
                        ))
fig.update_layout(
                  title="Battery discharge capacity",
                  xaxis_title="Working time [hours]",
                  yaxis_title=f"Battery capacity in Ahr"
    )
fig.update_layout({'plot_bgcolor': '#f2f8fd',
                   'paper_bgcolor': 'white',}, 
                    template='plotly_white')

## 2. Define a physical model

In [6]:
from math import e
L = (dfb['Capacity']-dfb['Capacity'].iloc[0:1].values[0])/-dfb['Capacity'].iloc[0:1].values[0]
K = 0.13
L_1 = 1-e**(-K*dfb.index*dfb['Temperature_measured']/(dfb['Time']))
dfb['C. Capacity'] = -(L_1*dfb['Capacity'].iloc[0:1].values[0]) + dfb['Capacity'].iloc[0:1].values[0]

In [7]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=dfb['C. Capacity'],
                         mode='lines',
                         name='Physical model',
                         line=dict(color='navy', 
                                   width=2.5,
                                   )))

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=dfb['Capacity'],
                         mode='markers',
                         marker=dict(
                              size=4,
                              color='grey',
                              symbol='cross'
                                 ),
                         name='NASA dataset',
                         line_color='navy'))
fig.update_layout(
    title="Physical model comparison ",
    xaxis_title="Cycles",
    yaxis_title="𝐶, Capacity [Ahr]")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.9,
    xanchor="left",
    x=0.8
))

fig.update_layout({'plot_bgcolor': '#f2f8fd',
                  'paper_bgcolor': 'white',}, 
                   template='plotly_white')

## 3. Compare experimental data with physical model

In [8]:
# Mean Absolute Error
M = pd.DataFrame()
S = pd.DataFrame()
def MAE(M,S):    
    return np.sum(S-M)/len(S)

print(f'Mean Absolute Error =', round(MAE(dfb['Capacity'], dfb['C. Capacity']), 3))

Mean Absolute Error = 0.004


## 4. Hybrid digital twin

In [9]:
#Define inputs and outputs
X_in = dfb['C. Capacity']          # input: the simulation time series
X_out = dfb['Capacity'] - dfb['C. Capacity']   # output: difference between measurement and simulation

X_in_train, X_in_test, X_out_train, X_out_test = train_test_split(X_in, X_out, test_size=0.33)

In [10]:
X_in_train.shape

(112,)

In [11]:
#The Dense function in Keras constructs a fully connected neural network layer, automatically initializing the weights as biases.
#First hidden layer
model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

In [12]:
epochs = 100
loss = "mse"
model.compile(optimizer='adam',
              loss=loss,
              metrics=['mae'], #Mean Absolute Error
             )
history = model.fit(X_in_train, X_out_train, 
                    shuffle=True, 
                    epochs=epochs,
                    batch_size=20,
                    validation_data=(X_in_test, X_out_test), 
                    verbose=1)

Epoch 1/100
6/6 [==============================] - 15s 80ms/step - loss: 0.0036 - mae: 0.0493 - val_loss: 7.4897e-04 - val_mae: 0.0225
Epoch 2/100
6/6 [==============================] - 0s 4ms/step - loss: 0.0014 - mae: 0.0306 - val_loss: 7.5693e-04 - val_mae: 0.0232
Epoch 3/100
6/6 [==============================] - 0s 5ms/step - loss: 9.8683e-04 - mae: 0.0249 - val_loss: 7.5456e-04 - val_mae: 0.0217
Epoch 4/100
6/6 [==============================] - 0s 4ms/step - loss: 8.0911e-04 - mae: 0.0245 - val_loss: 8.0746e-04 - val_mae: 0.0245
Epoch 5/100
6/6 [==============================] - 0s 5ms/step - loss: 0.0010 - mae: 0.0277 - val_loss: 8.0806e-04 - val_mae: 0.0214
Epoch 6/100
6/6 [==============================] - 0s 4ms/step - loss: 0.0011 - mae: 0.0277 - val_loss: 7.5055e-04 - val_mae: 0.0233
Epoch 7/100
6/6 [==============================] - 0s 4ms/step - loss: 9.1094e-04 - mae: 0.0259 - val_loss: 7.2945e-04 - val_mae: 0.0220
Epoch 8/100
6/6 [==============================] - 0s 4

In [13]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(0, epochs, 1),
                         y=history.history['mae'],
                         mode='lines',
                         name=f'Training MAE',
                         marker_size=3, 
                         line_color='orange'))
fig.add_trace(go.Scatter(x=np.arange(0, epochs, 1),
                         y=history.history['val_mae'],
                         mode='lines',
                         name=f'Validation MAE',
                         line_color='grey'))

fig.update_layout(
                  title="Network training",
                  xaxis_title="Epochs",
                  yaxis_title=f"Mean Absolute Error")
fig.update_layout({'plot_bgcolor': '#f2f8fd' , 
                   'paper_bgcolor': 'white',}, 
                   template='plotly_white')

## 4. Compile the hybrid digital twin

In [14]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_in_train, 
                         y=X_out_train,
                         mode='markers',
                         name=f'Modelled Capacity',
                         marker=dict(
                              size=4,
                              color='grey',
                              symbol='cross'
                                 ), 
                        line_color='crimson'))
fig.add_trace(go.Scatter(x = X_in_train, 
                         y=model.predict(X_in_train).reshape(-1),
                         mode='lines',
                         name=f'Trained Capacity',
                         line=dict(color='navy', width=3)))
fig.update_layout(
    title="Network training",
    xaxis_title="Modelled capacity",
    yaxis_title="Δ (Mod. Capacity - Measured Cap.)")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.45
))
fig.update_layout({'plot_bgcolor': '#f2f8fd' , #or azure
'paper_bgcolor': 'white',}, template='plotly_white')

In [15]:
X_twin = X_in + model.predict(X_in).reshape(-1)

fig = go.Figure()

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=X_twin,
                         mode='lines',
                         name=f'Hybrid digial twin',
                         line=dict(color='firebrick', width=3)))
fig.add_trace(go.Scatter(x=dfb.index, 
                         y=dfb['C. Capacity'],
                         mode='lines',
                         name=f'Modelled capacity',
                         line=dict(color='navy', 
                                   width=3,
                                   dash='dash')))

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=dfb['Capacity'],
                         mode='markers',
                         marker=dict(
                              size=4,
                              color='grey',
                              symbol='cross'
                                 ),
                         name=f'Observed capacity',
                         line_color='navy'))
fig.update_layout(
    title="Comparison of hybrid twin with other models",
    xaxis_title="Cycles",
    yaxis_title="Capacity in Ahr")
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.77
))
fig.update_layout({'plot_bgcolor': '#f2f8fd',
                  'paper_bgcolor': 'white',}, 
                   template='plotly_white')

## 5. Prediction with hybrid twin model

In [16]:
cycles = np.arange(168,500,1)
temperature = dfb['Temperature_measured'].iloc[167]
time = dfb['Time'].iloc[167]
K = 0.13
L_e = 1-e**(-K*cycles*temperature/time)
X_in_e = -(L_e*dfb['Capacity'].iloc[0:1].values[0]) + dfb['Capacity'].iloc[0:1].values[0]
C_twin_e = X_in_e + model.predict(X_in_e).reshape(-1)

In [17]:
X_twin = X_in + model.predict(X_in).reshape(-1)

fig = go.Figure()

fig.add_trace(go.Scatter(x=cycles, 
                         y=X_in_e,
                         mode='lines',
                         name=f'C modelled (predicted)',
                         line=dict(color='navy', 
                                   width=3,
                                   dash='dash')))
fig.add_trace(go.Scatter(x=cycles, 
                         y=C_twin_e,
                         mode='lines',
                         name=f'C Digital twin (predicted)',
                         line=dict(color='crimson', 
                                   width=3,
                                   dash='dash'
                                  )))

fig.add_trace(go.Scatter(x=dfb.index, 
                         y=X_twin,
                         mode='lines',
                         name=f'C Digital twin',
                         line=dict(color='crimson',
                                   width=2)))
fig.add_trace(go.Scatter(x=dfb.index, 
                         y=dfb['C. Capacity'],
                         mode='lines',
                         name=f'C modelled',
                         line=dict(color='navy', 
                                   width=2)))

fig.update_layout(
    title="Battery capacity prediction",
    xaxis_title="Cycles",
    yaxis_title="Battery capacity [Ahr]")
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.95,
    xanchor="left",
    x=0.72
))
fig.update_layout({'plot_bgcolor': '#f2f8fd',
                  'paper_bgcolor': 'white',}, 
                   template='plotly_white')